In [1]:
pip install pyquery

In [2]:
from urllib.parse import urlencode
from pyquery import PyQuery as py
import requests
import pandas as pd
import re

In [3]:
def get_information(since_id = '',uid = 0,):
    headers = {'Referer': 'https://m.weibo.cn',
                'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.85 Safari/537.36',
                'X-Requested-With': 'XMLHttpRequest'}
    params = {
                'uid': uid,
                'luicode': 10000011,
                'lfid': '100103type=1&q=?',
                'type': int(uid),
                'value': 5448226524,
                'containerid': '107603' + str(uid),
                'since_id': since_id
    }
    url = 'https://m.weibo.cn/api/container/getIndex?' + urlencode(params)
    response = requests.get(url,headers = headers)
    json_text = response.json()
    since_id = json_text.get('data').get('cardlistInfo').get('since_id')
    return json_text,since_id

In [4]:
def get_fulltext(mid = ''):
    headers = {'Referer': 'https://m.weibo.cn',
                'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.85 Safari/537.36',
                'X-Requested-With': 'XMLHttpRequest'}
    params = {
                'id':mid
    }
    url = 'https://m.weibo.cn/statuses/extend?' + urlencode(params)
    response = requests.get(url,headers = headers)
    json_text = response.json()
    print('---',json_text)
    if json_text.get('ok')==0:
        return ''
    else:
        text = re.sub("<[^>]*?>","",json_text.get('data').get('longTextContent'))
        return text

In [5]:
def parse_json(json):
    items = json.get('data').get('cards')
    for item in items:
        item = item.get('mblog')
        mid = item.get('id')
        weibo = {}
        weibo['Time'] = item.get('created_at')
        weibo['Text'] = get_fulltext(mid)
        yield weibo

In [6]:
uid = 3186551317
p = 50
s = get_information(since_id = '',uid = uid)

data = []
for i in range(p):
    s = get_information(since_id = s[1],uid = uid)
    for i in parse_json(s[0]):
        print(i)
        data.append(i)
pd.DataFrame(data)

--- {'ok': 1, 'data': {'ok': 1, 'longTextContent': '<a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E6%9C%80%E6%96%B0%E7%96%AB%E6%83%85%E9%80%9A%E6%8A%A5%23&extparam=%23%E6%9C%80%E6%96%B0%E7%96%AB%E6%83%85%E9%80%9A%E6%8A%A5%23" data-hide=""><span class="surl-text">#最新疫情通报#</span></a>【2021年5月6日广州市<a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E6%96%B0%E5%86%A0%E8%82%BA%E7%82%8E%23&extparam=%23%E6%96%B0%E5%86%A0%E8%82%BA%E7%82%8E%23" data-hide=""><span class="surl-text">#新冠肺炎#</span></a>疫情情况】2021年5月6日0时至24时，广州市新增境外输入确诊病例5例。新增出院病例6例。<br /><br />\u3000\u3000截至2021年5月6日24时，累计报告境外输入确诊病例782例、境外输入关联病例28例。累计出院765例，尚在院治疗45例。<br /><br />\u3000\u3000截至2021年5月6日24时，累计报告境内确诊病例349例。累计出院348例，累计死亡1例。<br /><br />\u3000\u30002021年5月6日0时至24时，新增境外输入无症状感染者6例。<a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E5%85%B3%E6%B3%A8%E6%96%B0%E5%86%A0%E8%82%BA%E7%82%8E%23&extparam=%23%E5%85%B3%E6%B3%A8%E6%96%B0%E5%86

--- {'ok': 1, 'data': {'ok': 1, 'longTextContent': '【<a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E4%B8%96%E7%95%8C%E6%89%8B%E5%8D%AB%E7%94%9F%E6%97%A5%23&isnewpage=1" data-hide=""><span class="surl-text">#世界手卫生日#</span></a>】这次疫情让我们认识到了保持手卫生的重要性。洗手可以帮助我们预防多种疾病，如何正确洗手？请查看视频！', 'reposts_count': 0, 'comments_count': 0, 'attitudes_count': 1}}
{'Time': 'Wed May 05 20:20:13 +0800 2021', 'Text': '【#世界手卫生日#】这次疫情让我们认识到了保持手卫生的重要性。洗手可以帮助我们预防多种疾病，如何正确洗手？请查看视频！'}
--- {'ok': 1, 'data': {'ok': 1, 'longTextContent': '<a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E5%AD%A6%E5%85%9A%E5%8F%B2%23" data-hide=""><span class="surl-text">#学党史#</span></a>【党领导人民在中华民族伟大复兴旗帜下阔步前进】<a data-url="http://t.cn/A6cgh91l" href="http://t.cn/A6cgh91l" data-hide=""><span class=\'url-icon\'><img style=\'width: 1rem;height: 1rem\' src=\'https://h5.sinaimg.cn/upload/2015/09/25/3/timeline_card_small_web_default.png\'></span><span class="surl-text">学党史 | 党领导

--- {'ok': 1, 'data': {'ok': 1, 'longTextContent': '<a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E6%9C%80%E6%96%B0%E7%96%AB%E6%83%85%E9%80%9A%E6%8A%A5%23&extparam=%23%E6%9C%80%E6%96%B0%E7%96%AB%E6%83%85%E9%80%9A%E6%8A%A5%23" data-hide=""><span class="surl-text">#最新疫情通报#</span></a>【2021年5月4日广东省<a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E6%96%B0%E5%86%A0%E8%82%BA%E7%82%8E%23&extparam=%23%E6%96%B0%E5%86%A0%E8%82%BA%E7%82%8E%23" data-hide=""><span class="surl-text">#新冠肺炎#</span></a>疫情情况】<a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E5%85%B3%E6%B3%A8%E6%96%B0%E5%86%A0%E8%82%BA%E7%82%8E%23&extparam=%23%E5%85%B3%E6%B3%A8%E6%96%B0%E5%86%A0%E8%82%BA%E7%82%8E%23" data-hide=""><span class="surl-text">#关注新冠肺炎#</span></a>  <a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E6%9C%80%E6%96%B0%E7%96%AB%E6%83%85%E5%9C%B0%E5%9B%BE%23&extparam=%23%E6%9C%80%E6%96%B

--- {'ok': 1, 'data': {'ok': 1, 'longTextContent': '<a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E6%9C%80%E6%96%B0%E7%96%AB%E6%83%85%E9%80%9A%E6%8A%A5%23&extparam=%23%E6%9C%80%E6%96%B0%E7%96%AB%E6%83%85%E9%80%9A%E6%8A%A5%23" data-hide=""><span class="surl-text">#最新疫情通报#</span></a>【2021年5月3日广东省<a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E6%96%B0%E5%86%A0%E8%82%BA%E7%82%8E%23&extparam=%23%E6%96%B0%E5%86%A0%E8%82%BA%E7%82%8E%23" data-hide=""><span class="surl-text">#新冠肺炎#</span></a>疫情情况】<a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E5%85%B3%E6%B3%A8%E6%96%B0%E5%86%A0%E8%82%BA%E7%82%8E%23&extparam=%23%E5%85%B3%E6%B3%A8%E6%96%B0%E5%86%A0%E8%82%BA%E7%82%8E%23" data-hide=""><span class="surl-text">#关注新冠肺炎#</span></a>  <a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E6%9C%80%E6%96%B0%E7%96%AB%E6%83%85%E5%9C%B0%E5%9B%BE%23&extparam=%23%E6%9C%80%E6%96%B

--- {'ok': 1, 'data': {'ok': 1, 'longTextContent': '<a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E6%8F%90%E9%86%92%23&isnewpage=1" data-hide=""><span class="surl-text">#提醒#</span></a>【出游时个人防护怎么做？什么时候一定要戴口罩？】<a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E4%BA%94%E4%B8%80%23&isnewpage=1" data-hide=""><span class="surl-text">#五一#</span></a>出游时如何落实个人防护，如在公共交通上、旅游景区等的防护措施？广州市疾控中心副主任张周斌这样说！<a data-url="http://t.cn/A6cuvyuu" href="http://t.cn/A6cuvyuu" data-hide=""><span class=\'url-icon\'><img style=\'width: 1rem;height: 1rem\' src=\'https://h5.sinaimg.cn/upload/2015/09/25/3/timeline_card_small_web_default.png\'></span><span class="surl-text">网页链接</span></a> <a data-url="http://t.cn/A6cuvy1t" href="https://video.weibo.com/show?fid=1034:4632007574224981" data-hide=""><span class=\'url-icon\'><img style=\'width: 1rem;height: 1rem\' src=\'https://h5.sinaimg.cn/upload/2015/09/25/3/timeline_card_small_video_default.png\'></s

...

--- {'ok': 1, 'data': {'ok': 1, 'longTextContent': '<a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E6%9C%80%E6%96%B0%E7%96%AB%E6%83%85%E9%80%9A%E6%8A%A5%23&extparam=%23%E6%9C%80%E6%96%B0%E7%96%AB%E6%83%85%E9%80%9A%E6%8A%A5%23" data-hide=""><span class="surl-text">#最新疫情通报#</span></a>【2021年2月22日广东省<a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E6%96%B0%E5%86%A0%E8%82%BA%E7%82%8E%23&extparam=%23%E6%96%B0%E5%86%A0%E8%82%BA%E7%82%8E%23" data-hide=""><span class="surl-text">#新冠肺炎#</span></a>疫情情况】<a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E5%85%B3%E6%B3%A8%E6%96%B0%E5%86%A0%E8%82%BA%E7%82%8E%23&extparam=%23%E5%85%B3%E6%B3%A8%E6%96%B0%E5%86%A0%E8%82%BA%E7%82%8E%23" data-hide=""><span class="surl-text">#关注新冠肺炎#</span></a>  <a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E6%9C%80%E6%96%B0%E7%96%AB%E6%83%85%E5%9C%B0%E5%9B%BE%23&extparam=%23%E6%9C%80%E6%96%

--- {'ok': 1, 'data': {'ok': 1, 'longTextContent': '<a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E9%98%B2%E7%96%AB%E6%94%BF%E7%AD%96%23&extparam=%23%E9%98%B2%E7%96%AB%E6%94%BF%E7%AD%96%23" data-hide=""><span class="surl-text">#防疫政策#</span></a>【春节后返穗来穗，这些事要记住！】<a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E6%98%A5%E8%8A%82%23&isnewpage=1" data-hide=""><span class="surl-text">#春节#</span></a>假期结束，返穗、来穗人员请留意做好以下疫情防控措施↓↓↓  <a data-url="http://t.cn/A6tfOPDI" href="https://video.weibo.com/show?fid=1034:4606721415381029" data-hide=""><span class=\'url-icon\'><img style=\'width: 1rem;height: 1rem\' src=\'https://h5.sinaimg.cn/upload/2015/09/25/3/timeline_card_small_video_default.png\'></span><span class="surl-text">健康广州的微博视频</span></a> ', 'reposts_count': 1, 'comments_count': 0, 'attitudes_count': 3}}
{'Time': 'Sat Feb 20 17:04:13 +0800 2021', 'Text': '#防疫政策#【春节后返穗来穗，这些事要记住！】#春节#假期结束，返穗、来穗人员请留意做好以下疫情防控措施↓↓↓  健康广州的微博视频 '}
-

--- {'ok': 1, 'data': {'ok': 1, 'longTextContent': '【低头15/30/45/60度，颈椎分别要承重多少斤<span class="url-icon"><img alt=[泪] src="https://h5.sinaimg.cn/m/emoticon/icon/default/d_lei-4cdf6ee412.png" style="width:1em; height:1em;" /></span>】低头玩手机，已经成为不少人平时最常做的姿势。欧洲脊柱协会曾发出警示：<a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E9%95%BF%E6%9C%9F%E4%BD%8E%E5%A4%B4%E7%8E%A9%E6%89%8B%E6%9C%BA%E7%9B%B8%E5%BD%93%E4%BA%8E%E8%84%96%E5%AD%90%E8%B4%9F%E9%87%8D50%E6%96%A4%23&extparam=%23%E9%95%BF%E6%9C%9F%E4%BD%8E%E5%A4%B4%E7%8E%A9%E6%89%8B%E6%9C%BA%E7%9B%B8%E5%BD%93%E4%BA%8E%E8%84%96%E5%AD%90%E8%B4%9F%E9%87%8D50%E6%96%A4%23" data-hide=""><span class="surl-text">#长期低头玩手机相当于脖子负重50斤#</span></a>。①前倾呈15°角，颈椎承重大概为24斤；②前倾呈30°角，颈椎承重为36斤；③前倾呈45°角，颈椎承重为44斤；④前倾呈60°角时，颈椎承重可达54斤。玩手机的正确姿势：腰背挺直，将手机举起与眼睛的高度一致，视线与手机屏幕垂直，下巴不低垂也不仰起。via 生命时报', 'reposts_count': 0, 'comments_count': 0, 'attitudes_count': 6}}
{'Time': 'Thu Feb 18 14:51:18 +0800 2021', 'Text': '【低头15/30/45/60度，颈椎分别要承重多少斤】低头玩手机，已经成为不少人平时最

--- {'ok': 1, 'data': {'ok': 1, 'longTextContent': '<a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E6%9C%80%E6%96%B0%E7%96%AB%E6%83%85%E9%80%9A%E6%8A%A5%23&extparam=%23%E6%9C%80%E6%96%B0%E7%96%AB%E6%83%85%E9%80%9A%E6%8A%A5%23" data-hide=""><span class="surl-text">#最新疫情通报#</span></a>【2021年2月17日广东省<a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E6%96%B0%E5%86%A0%E8%82%BA%E7%82%8E%23&extparam=%23%E6%96%B0%E5%86%A0%E8%82%BA%E7%82%8E%23" data-hide=""><span class="surl-text">#新冠肺炎#</span></a>疫情情况】<a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E5%85%B3%E6%B3%A8%E6%96%B0%E5%86%A0%E8%82%BA%E7%82%8E%23&extparam=%23%E5%85%B3%E6%B3%A8%E6%96%B0%E5%86%A0%E8%82%BA%E7%82%8E%23" data-hide=""><span class="surl-text">#关注新冠肺炎#</span></a>  <a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E6%9C%80%E6%96%B0%E7%96%AB%E6%83%85%E5%9C%B0%E5%9B%BE%23&extparam=%23%E6%9C%80%E6%96%

,Time,Text
0,Fri May 07 11:15:03 +0800 2021,#最新疫情通报#【2021年5月6日广州市#新冠肺炎#疫情情况】2021年5月6日0时至24...
1,Fri May 07 09:04:20 +0800 2021,#最新疫情通报#【2021年5月7日广东省#新冠肺炎#疫情情况】#关注新冠肺炎# #最新疫...
2,Thu May 06 12:05:03 +0800 2021,#援藏#【“雪域江南”守护妇孺安康，这群党员医护千里送医】援藏 | “雪域江南”守护妇孺安康...
3,Thu May 06 11:55:03 +0800 2021,#广州健康码上线木棉花皮肤#【广州穗康码“变”了，打完码上送“花”！】惊艳 | 广州穗康码“...
4,Thu May 06 11:35:03 +0800 2021,#学党史#【传承先辈精神 坚定理想信念（人民论坛）】学党史 | 传承先辈精神 坚定理想信念（...
...,...,...
487,Tue Feb 16 11:15:03 +0800 2021,#最新疫情通报#【2021年2月15日广州市#新冠肺炎#疫情情况】2021年2月15日0时至...
488,Tue Feb 16 09:14:24 +0800 2021,#最新疫情通报#【2021年2月16日广东省#新冠肺炎#疫情情况】#关注新冠肺炎# #最新...
489,Tue Feb 16 09:12:37 +0800 2021,#别样中国年#【正月#初五#，转发#迎财神#！】今天初五，年俗多多：①迎财神：过了初一，最重...
490,Mon Feb 15 12:40:12 +0800 2021,#提醒#【假期#喝酒#攻略，能不能喝、怎么喝不伤身？看这一篇就够了！】提醒 | 假期喝酒攻略...


In [43]:
pd.DataFrame(data).to_csv('健康广州.csv', encoding='utf_8_sig')